# 🌟 XL Lora Trainer by Hollowstrawberry

❗ **Colab Premium is recommended.** Ideally you'd be changing the runtime to an A100 and use the maximum batch size.  
However you can still train for free if you load a diffusers model, it will just take much longer.  


This colab is based on the work of [Kohya-ss](https://github.com/kohya-ss/sd-scripts) and [Linaqruf](https://github.com/Linaqruf/kohya-trainer). Thank you!

### ⭕ Disclaimer
The purpose of this document is to research bleeding-edge technologies in the field of machine learning inference.  
Please read and follow the [Google Colab guidelines](https://research.google.com/colaboratory/faq.html) and its [Terms of Service](https://research.google.com/colaboratory/tos_v3.html).

| |GitHub|🇬🇧 English|🇪🇸 Spanish|
|:--|:-:|:-:|:-:|
| 🏠 **Homepage** | [![GitHub](https://raw.githubusercontent.com/hollowstrawberry/kohya-colab/main/assets/github.svg)](https://github.com/hollowstrawberry/kohya-colab) | | |
| 📊 **Dataset Maker** | [![GitHub](https://raw.githubusercontent.com/hollowstrawberry/kohya-colab/main/assets/github.svg)](https://github.com/hollowstrawberry/kohya-colab/blob/main/Dataset_Maker.ipynb) | [![Open in Colab](https://raw.githubusercontent.com/hollowstrawberry/kohya-colab/main/assets/colab-badge.svg)](https://colab.research.google.com/github/hollowstrawberry/kohya-colab/blob/main/Dataset_Maker.ipynb) | [![Abrir en Colab](https://raw.githubusercontent.com/hollowstrawberry/kohya-colab/main/assets/colab-badge-spanish.svg)](https://colab.research.google.com/github/hollowstrawberry/kohya-colab/blob/main/Spanish_Dataset_Maker.ipynb) |
| ⭐ **Lora Trainer** | [![GitHub](https://raw.githubusercontent.com/hollowstrawberry/kohya-colab/main/assets/github.svg)](https://github.com/hollowstrawberry/kohya-colab/blob/main/Lora_Trainer.ipynb) | [![Open in Colab](https://raw.githubusercontent.com/hollowstrawberry/kohya-colab/main/assets/colab-badge.svg)](https://colab.research.google.com/github/hollowstrawberry/kohya-colab/blob/main/Lora_Trainer.ipynb) | [![Abrir en Colab](https://raw.githubusercontent.com/hollowstrawberry/kohya-colab/main/assets/colab-badge-spanish.svg)](https://colab.research.google.com/github/hollowstrawberry/kohya-colab/blob/main/Spanish_Lora_Trainer.ipynb) |
| 🌟 **XL Lora Trainer** | [![GitHub](https://raw.githubusercontent.com/hollowstrawberry/kohya-colab/main/assets/github.svg)](https://github.com/hollowstrawberry/kohya-colab/blob/main/Lora_Trainer_XL.ipynb) | [![Open in Colab](https://raw.githubusercontent.com/hollowstrawberry/kohya-colab/main/assets/colab-badge.svg)](https://colab.research.google.com/github/hollowstrawberry/kohya-colab/blob/main/Lora_Trainer_XL.ipynb) |  |

In [2]:
import os
import re
import toml
from time import time
from huggingface_hub import HfFileSystem
from IPython.display import Markdown, display
import ipywidgets as widgets

# These carry information from past executions
if "model_url" in globals():
  old_model_url = model_url
else:
  old_model_url = None
if "dependencies_installed" not in globals():
  dependencies_installed = False
if "model_file" not in globals():
  model_file = None

# These may be set by other cells, some are legacy
if "custom_dataset" not in globals():
  custom_dataset = None
if "override_dataset_config_file" not in globals():
  override_dataset_config_file = None
if "override_config_file" not in globals():
  override_config_file = None  

COLAB = False
SOURCE = "https://github.com/kohya-ss/sd-scripts"
TAG = "tags/v0.8.7"
COMMIT = None
BETTER_EPOCH_NAMES = True
LOAD_TRUNCATED_IMAGES = True
try:
  LOWRAM = int(next(line.split()[1] for line in open('/proc/meminfo') if "MemTotal" in line)) / (1024**2) < 15
except:
  LOWRAM = False

display(
    widgets.HTML("<h1>🚩 Start Here</h1>"),
)

project_name_widget = widgets.Text(description='Project name:')
folder_structure_widget = widgets.Dropdown(
    options=[
        "Organize by category (MyDrive/lora_training/datasets/project_name)", 
        "Organize by project (MyDrive/Loras/project_name/dataset)"
    ],
    value="Organize by project (MyDrive/Loras/project_name/dataset)",
    description='Folder structure:',
    disabled=False,
)

training_model_widget = widgets.Dropdown(
    options=["Pony Diffusion V6 XL", "Animagine XL V3", "Stable Diffusion XL 1.0 base"],
    value="Pony Diffusion V6 XL",
    description='Training model:',
    disabled=False,
)

optional_custom_training_model_url_widget = widgets.Text(
    description = 'Optional: custom training model url'
)

load_diffusers_widget = widgets.Checkbox(
    value=True,
    description='Load diffusers',
    disabled=False,
    indent=False
)

wandb_key_widget = widgets.Text(description='Wandb key:')

display(
    widgets.HTML("<h2>▶️ Setup</h2>"),
    widgets.Label("Your project name will be the same as the folder containing your images. Spaces aren't allowed."),
    project_name_widget,
    widgets.Label("The folder structure doesn't matter and is purely for comfort. Make sure to always pick the same one. I like organizing by project."),
    folder_structure_widget,
    widgets.Label(" Decide the model that will be downloaded and used for training. You can also choose your own by pasting its download link."),
    training_model_widget,
    optional_custom_training_model_url_widget,
    widgets.Label("Using a diffusers model utilizes less resources. The 3 options above will work with or without diffusers."),
    load_diffusers_widget,
    widgets.Label("Use wandb if you want to visualize the progress of your training over time."),
    wandb_key_widget
)

display(widgets.HTML('<h2> ▶️ Processing </h2>'))

resolution_widget = widgets.BoundedIntText(
    value=1024,
    min=768,
    max=1536,
    step=128,
    description='Resolution:',
    disabled=False
)

caption_extension_widget = widgets.Text(
    value=".txt",
    description="Caption extension"
)

shuffle_tags_widget = widgets.Checkbox(
    value=True,
    description='Shuffle tags',
    disabled=False,
    indent=False
)

activation_tags_widget = widgets.Dropdown(
    options= ['0', '1' , '2' ,'3'],
    value='1',
    description='Activation tags:',
    disabled=False,
)

display(
    resolution_widget,
    caption_extension_widget,
    widgets.Label('Shuffling anime tags in place improves learning and prompting. An activation tag goes at the start of every text file and will not be shuffled.'),
    shuffle_tags_widget,
    activation_tags_widget,
)

display(widgets.HTML('<h2>▶️ Steps</h2>'))

num_repeats_widget = widgets.IntText(
    value=2,
    description='# repeats:',
    disabled=False
)

preferred_unit_widget = widgets.RadioButtons(
    options=["Epochs", "Steps"],
    value='Epochs', 
    description='Preferred unit',
    disabled=False
)

how_many_widget = widgets.IntText(
    value=10,
    description='How many:',
    disabled=False
)

save_every_n_epochs_widget = widgets.IntText(
    value=1,
    description='Saving every N epochs:',
    disabled=False
)

keep_only_last_n_epochs_widget = widgets.IntText(
    value=10,
    description='Keep only last N epochs:',
    disabled=False
)

display(
    widgets.Label('Your images will repeat this number of times during training. I recommend that your images multiplied by their repeats is between 200 and 400.'),
    num_repeats_widget,
    widgets.Label('Choose how long you want to train for. A good starting point is around 10 epochs or around 2000 steps.'),
    widgets.Label('One epoch is a number of steps equal to: your number of images multiplied by their repeats, divided by batch size.'),
    preferred_unit_widget,
    how_many_widget,
    widgets.Label("Saving more epochs will let you compare your Lora's progress better."),
    save_every_n_epochs_widget,
    keep_only_last_n_epochs_widget,
)

unet_lr_widget = widgets.FloatText(
    value=3e-4,
    description='UNet LR:',
    disabled=False
)

text_encoder_lr_widget = widgets.FloatText(
    value=6e-5,
    description='Text encoder LR:',
    disabled=False
)

lr_scheduler_widget = widgets.Dropdown(
    options=["constant", "cosine", "cosine_with_restarts", "constant_with_warmup", "linear", "polynomial"],
    value='cosine_with_restarts',
    description='LR Scheduler:',
    disabled=False,
)

lr_scheduler_number_widget = widgets.IntText(
    value=3,
    description='LR Scheduler #:',
    disabled=False
)

lr_warmup_ratio_widget = widgets.FloatSlider(
    value=0.05,
    min=0,
    max=0.2,
    step=0.01,
    description='LR Warmiup Ratio:',
    disabled=False,
    continuous_update=False,
    orientation='horizontal',
    readout=True,
    readout_format='.1f',
)

min_snr_gamma_widget = widgets.FloatSlider(
    value=8.0,
    min=4,
    max=16.0,
    step=0.5,
    description='Min SNR Gamma:',
    disabled=False,
    continuous_update=False,
    orientation='horizontal',
    readout=True,
    readout_format='.1f',
)

multinoise_widget = widgets.Checkbox(
    value=True,
    description='Multinoise',
    disabled=False,
    indent=False
)

display(
    widgets.HTML('<h2>▶️ Learning </h2>'),
    widgets.Label('The learning rate is the most important for your results. If you want to train slower with lots of images, or if your dim and alpha are high, move the unet to 2e-4 or lower.'),
    widgets.Label("The text encoder helps your Lora learn concepts slightly better. It is recommended to make it half or a fifth of the unet. If you're training a style you can even set it to 0."),
    unet_lr_widget,
    text_encoder_lr_widget,
    widgets.Label("The scheduler is the algorithm that guides the learning rate. If you're not sure, pick `constant` and ignore the number. I personally recommend `cosine_with_restarts` with 3 restarts."),
    lr_scheduler_widget,
    lr_scheduler_number_widget,
    widgets.Label('Steps spent "warming up" the learning rate during training for efficiency. I recommend leaving it at 5%.'),
    lr_warmup_ratio_widget,
    widgets.Label('Adjusts loss over time, makes learning much more efficient. The paper recommends 5.0, I recommend 8.0 for anime. Higher value makes it less strict. Set to 0 to disable.'),
    min_snr_gamma_widget,
    widgets.Label('Multinoise may help with color balance (darker darks, lighter lights).'),
    multinoise_widget
)

lora_type_widget = widgets.RadioButtons(
    options=["LoRA", "LoCon"],
    value='LoRA',
    description='LoRA type:',
    disabled=False
)

network_dim_widget = widgets.IntSlider(
    value=8,
    min=1,
    max=32,
    step=1,
    description='network_dim:',
    disabled=False,
    continuous_update=False,
    orientation='horizontal',
    readout=True,
    readout_format='d'
)

network_alpha_widget = widgets.IntSlider(
    value=4,
    min=1,
    max=32,
    step=1,
    description='network_alpha:',
    disabled=False,
    continuous_update=False,
    orientation='horizontal',
    readout=True,
    readout_format='d'
)

conv_dim_widget = widgets.IntSlider(
    value=4,
    min=1,
    max=32,
    step=1,
    description='conv_dim:',
    disabled=False,
    continuous_update=False,
    orientation='horizontal',
    readout=True,
    readout_format='d'
)

conv_alpha_widget = widgets.IntSlider(
    value=1,
    min=1,
    max=32,
    step=1,
    description='conv_alpha:',
    disabled=False,
    continuous_update=False,
    orientation='horizontal',
    readout=True,
    readout_format='d'
)

display(
    widgets.HTML('<h2>▶️ Structure</h2>'),
    lora_type_widget,
    widgets.Label('Below are some recommended XL values for the following settings:'),
    widgets.HTML("""
        <table>
            <thead>
                <tr>
                    <th>type</th>
                    <th>network_dim</th>
                    <th>network_alpha</th>
                    <th>conv_dim</th>
                    <th>conv_alpha</th>
                </tr>
            </thead>
            <tbody>
                <tr>
                    <th>Regular LoRA</th>
                    <td>8</td>
                    <td>4</td>
                    <td>-</td>
                    <td>-</td>
                </tr>
                <tr>
                    <th>Style LoCon</th>
                    <td>16</td>
                    <td>8</td>
                    <td>16</td>
                    <td>8<td>
                </tr>
            </tbody>
        <table>
    """),
    widgets.Label("More dim means larger Lora, it can hold more information but more isn't always better."),
    network_dim_widget,
    network_alpha_widget,
    widgets.Label("The following two values only apply to the additional layers of LoCon."),
    conv_dim_widget,
    conv_alpha_widget,
)

train_batch_size_widget = widgets.IntSlider(
    value=4,
    min=1,
    max=16,
    step=1,
    description='Train batch size:',
    disabled=False,
    continuous_update=False,
    orientation='horizontal',
    readout=True,
    readout_format='d'
)

cross_attention_widget = widgets.RadioButtons(
    options=["sdpa", "xformers"],
    value="sdpa",
    description='Cross attention:',
    disabled=False
)

mixed_precision_widget = widgets.RadioButtons(
    options=["bf16", "fp16"],
    value="fp16",
    description='Mixed precision:',
    disabled=False
)

cache_latents_widget = widgets.Checkbox(
    value=True,
    description='Cache latents.',
    disabled=False,
    indent=False
)

cache_latents_to_drive_widget = widgets.Checkbox(
    value=True,
    description='Cache latents to Drive.',
    disabled=False,
    indent=False
)

cache_text_encoder_outputs_widget = widgets.Checkbox(
    value=False,
    description='Cache text encoder outputs:',
    disabled=False,
    indent=False
)

display(
    widgets.HTML('<h2>▶️ Training</h2>'),
    widgets.Label('Adjust these parameters depending on your colab configuration.'),
    widgets.Label('''If you're using the free tier you should select a diffusers model at the top of this cell.'''),
    widgets.HTML('<hr>'),
    widgets.Label('Higher batch size is often faster but uses more memory.'),
    train_batch_size_widget,
    widgets.Label('I have found no substantial difference between sdpa and xformers.'),
    cross_attention_widget,
    widgets.Label('''If you're on an A100 you should enable bf16.'''),
    mixed_precision_widget,
    widgets.Label('Caching latents to drive will add a 250KB file next to each image but will use considerably less memory.'),
    cache_latents_widget,
    cache_latents_to_drive_widget,
    widgets.Label('The following option will turn off shuffle_tags and disable text encoder training.'),
    cache_text_encoder_outputs_widget
)

optimizer_widget = widgets.Dropdown(
    options=["AdamW8bit", "Prodigy", "DAdaptation", "DadaptAdam", "DadaptLion", "AdamW", "Lion", "SGDNesterov", "SGDNesterov8bit", "AdaFactor"],
    value="AdamW8bit",
    description='Optimizer:',
    disabled=False,
)

optimizer_args_widget = widgets.Text(
    description="Optimizer args:",
    value="weight_decay=0.1 betas=[0.9,0.99]"
)

recommended_values_widget = widgets.Checkbox(
    value=True,
    description='Recommended values',
    disabled=False,
    indent=False
)

display(
    widgets.HTML('<h2>▶️ Advanced</h2>'),
    widgets.Label('The optimizer is the algorithm used for training. AdanW8Bit is the default and works great, while Prodigy manages learning rate automatically and may have several advantages such as training faster due to needing less steps as well as working better for small datasets.'),
    optimizer_widget,
    widgets.HTML('''
        Recommended args for:
        <ul>
            <li><b>AdamW8bit</b>: <div style='font-family: monospace;'>weight_decay=0.1 betas=[0.9,0.99]</div></li>
            <li><b>Prodigy</b>: <div style='font-family: monospace;'>decouple=True weight_decay=0.01 betas=[0.9,0.999] d_coef=2 use_bias_correction=True safeguard_warmup=True</div></li>
            <li><b>CAME</b>: <div style='font-family: monospace;'>weight_decay=0.04</div></li>
        </ul>
    '''),
    optimizer_args_widget,
    widgets.HTML('''
        <p> If <b>Dadapt</b> or <b>Prodigy</b> are selected and the recommended box is checked, the following recommended values will override any previous settings:
        <div style='font-family: monospace;'>unet_lr=0.75, text_encoder_lr=0.75, network_alpha=network_dim</div></p>
        <p> If <b>CAME + REX</b> is selected and the recommended box is checked, the following recommended values will override any previous settings:
        <div style='font-family: monospace;'>unet_lr=1e-4, text_encoder_lr=1e-6, betas=default_came_betas</div></p>
    '''),
    recommended_values_widget
)    

HTML(value='<h1>🚩 Start Here</h1>')

HTML(value='<h2>▶️ Setup</h2>')

Label(value="Your project name will be the same as the folder containing your images. Spaces aren't allowed.")

Text(value='', description='Project name:')

Label(value="The folder structure doesn't matter and is purely for comfort. Make sure to always pick the same …

...

Dropdown(description='Optimizer:', options=('AdamW8bit', 'Prodigy', 'DAdaptation', 'DadaptAdam', 'DadaptLion',…

HTML(value="\n        Recommended args for:\n        <ul>\n            <li><b>AdamW8bit</b>: <div style='font-…

Text(value='weight_decay=0.1 betas=[0.9,0.99]', description='Optimizer args:')

HTML(value="\n        <p> If <b>Dadapt</b> or <b>Prodigy</b> are selected and the recommended box is checked, …

Checkbox(value=True, description='Recommended values', indent=False)

In [9]:
# 👩‍💻 Cool code goes here

import pathlib
from accelerate.utils import write_basic_config

def install_dependencies(root_dir, repo_dir):
    global project_name, folder_structure, training_model, optional_custom_training_model_url
    global training_model, training_model, model_url, vae_file, model_file, resolution, caption_extension
    global shuffle_tags, shuffle_caption, activation_tags
    global num_repeats, preferred_unit, how_many, max_train_epochs, max_train_steps
    global max_train_epochs, max_train_steps, preferred_unit, how_many
    global save_every_n_epochs, keep_only_last_n_epochs, unet_lr, text_encoder_lr
    global lr_scheduler, lr_scheduler_number, lr_warmup_ratio, lr_warmup_steps
    global min_snr_gamma, multinoise, lora_type, network_dim, conv_alpha
    global network_module, network_args, train_batch_size, cross_attention 
    global mixed_precision, cache_latents, cache_latents_to_drive
    global cache_text_encoder_outputs, optimizer, optimizer_args, recommended_values
    global images_folder

    os.chdir(root_dir)
    global SOURCE
    !git clone {SOURCE} {repo_dir}
    os.chdir(repo_dir)
    global TAG
    if TAG:
        !git checkout {TAG}
        !git reset --hard
            
    !wget https://raw.githubusercontent.com/hollowstrawberry/kohya-colab/main/train_network_xl_wrapper.py -q -O train_network_xl_wrapper.py
    !wget https://raw.githubusercontent.com/hollowstrawberry/kohya-colab/main/dracula.py -q -O dracula.py

    !pip install -r requirements.txt
            
    #if cross_attention == "xformers":
    #    !pip install -q xformers==0.0.26.dev778
    if "CAME" in optimizer:
        !pip install came-pytorch
        !wget https://raw.githubusercontent.com/hollowstrawberry/kohya-colab/main/train_util.py -q -O library/train_util.py

    # patch kohya for minor stuff
    if LOWRAM:
        !sed -i "s@cpu@cuda@" library/model_util.py
    if LOAD_TRUNCATED_IMAGES:
        !sed -i 's/from PIL import Image/from PIL import Image, ImageFile\nImageFile.LOAD_TRUNCATED_IMAGES=True/g' library/train_util.py # fix truncated jpegs error
    if BETTER_EPOCH_NAMES:
        !sed -i 's/{:06d}/{:02d}/g' library/train_util.py # make epoch names shorter
        !sed -i 's/"." + args.save_model_as)/"-{:02d}.".format(num_train_epochs) + args.save_model_as)/g' train_network.py # name of the last epoch will match the rest

    if not os.path.exists(accelerate_config_file):
        write_basic_config(save_location=accelerate_config_file)

    os.environ["TF_CPP_MIN_LOG_LEVEL"] = "3"
    os.environ["BITSANDBYTES_NOWELCOME"] = "1"
    os.environ["SAFETENSORS_FAST_GPU"] = "1"

def validate_dataset():
    global lr_warmup_steps, lr_warmup_ratio, caption_extension, keep_tokens
    global project_name, folder_structure, training_model, optional_custom_training_model_url
    global training_model, training_model, model_url, vae_file, model_file, resolution, caption_extension
    global shuffle_tags, shuffle_caption, activation_tags
    global num_repeats, preferred_unit, how_many, max_train_epochs, max_train_steps
    global max_train_epochs, max_train_steps, preferred_unit, how_many
    global save_every_n_epochs, keep_only_last_n_epochs, unet_lr, text_encoder_lr
    global lr_scheduler, lr_scheduler_number, lr_warmup_ratio, lr_warmup_steps
    global min_snr_gamma, multinoise, lora_type, network_dim, conv_alpha
    global network_module, network_args, train_batch_size, cross_attention 
    global mixed_precision, cache_latents, cache_latents_to_drive
    global cache_text_encoder_outputs, optimizer, optimizer_args, recommended_values
    global images_folder
    
    supported_types = (".png", ".jpg", ".jpeg", ".webp", ".bmp")

    print("\n💿 Checking dataset...")
    if not project_name.strip() or any(c in project_name for c in " .()\"'\\/"):
        print("💥 Error: Please choose a valid project name.")
        return

    if custom_dataset:
        try:
            datconf = toml.loads(custom_dataset)
            datasets = [d for d in datconf["datasets"][0]["subsets"]]
        except:
            print(f"💥 Error: Your custom dataset is invalid or contains an error! Please check the original template.")
            return
        reg = [d.get("image_dir") for d in datasets if d.get("is_reg", False)]
        datasets_dict = {d["image_dir"]: d["num_repeats"] for d in datasets}
        folders = datasets_dict.keys()
        files = [f for folder in folders for f in os.listdir(folder)]
        images_repeats = {folder: (len([f for f in os.listdir(folder) if f.lower().endswith(supported_types)]), datasets_dict[folder]) for folder in folders}
    else:
        reg = []
        folders = [images_folder]
        files = os.listdir(images_folder)
        images_repeats = {images_folder: (len([f for f in files if f.lower().endswith(supported_types)]), num_repeats)}

    for folder in folders:
        if not os.path.exists(folder):
            print(f"💥 Error: The folder {folder.replace('/content/drive/', '')} doesn't exist.")
            return
    for folder, (img, rep) in images_repeats.items():
        if not img:
            print(f"💥 Error: Your {folder.replace('/content/drive/', '')} folder is empty.")
            return
    for f in files:
        if not f.lower().endswith((".txt", ".npz")) and not f.lower().endswith(supported_types):
            print(f"💥 Error: Invalid file in dataset: \"{f}\". Aborting.")
            return

    if not [txt for txt in files if txt.lower().endswith(".txt")]:
        caption_extension = ""

    pre_steps_per_epoch = sum(img*rep for (img, rep) in images_repeats.values())
    steps_per_epoch = pre_steps_per_epoch/train_batch_size
    total_steps = max_train_steps or int(max_train_epochs*steps_per_epoch)
    estimated_epochs = int(total_steps/steps_per_epoch)
    lr_warmup_steps = int(total_steps*lr_warmup_ratio)

    for folder, (img, rep) in images_repeats.items():
        print("📁"+folder.replace("/content/drive/", "") + (" (Regularization)" if folder in reg else ""))
        print(f"📈 Found {img} images with {rep} repeats, equaling {img*rep} steps.")
    print(f"📉 Divide {pre_steps_per_epoch} steps by {train_batch_size} batch size to get {steps_per_epoch} steps per epoch.")
    if max_train_epochs:
        print(f"🔮 There will be {max_train_epochs} epochs, for around {total_steps} total training steps.")
    else:
        print(f"🔮 There will be {total_steps} steps, divided into {estimated_epochs} epochs and then some.")

    if total_steps > 10000:
        print("💥 Error: Your total steps are too high. You probably made a mistake. Aborting...")
        return

    return True

def create_config(images_folder, output_folder, log_folder):
    global dataset_config_file, config_file, model_file
    global project_name, folder_structure, training_model, optional_custom_training_model_url
    global training_model, training_model, model_url, vae_file, model_file, resolution, caption_extension
    global shuffle_tags, shuffle_caption, activation_tags
    global num_repeats, preferred_unit, how_many, max_train_epochs, max_train_steps
    global max_train_epochs, max_train_steps, preferred_unit, how_many
    global save_every_n_epochs, keep_only_last_n_epochs, unet_lr, text_encoder_lr
    global lr_scheduler, lr_scheduler_number, lr_warmup_ratio, lr_warmup_steps
    global min_snr_gamma, multinoise, lora_type, network_dim, conv_alpha
    global network_module, network_args, train_batch_size, cross_attention 
    global mixed_precision, cache_latents, cache_latents_to_drive
    global cache_text_encoder_outputs, optimizer, optimizer_args, recommended_values

    if override_config_file:
        config_file = override_config_file
        print(f"\n⭕ Using custom config file {config_file}")
    else:
        config_dict = {
            "network_arguments": {
                "unet_lr": unet_lr,
                "text_encoder_lr": text_encoder_lr if not cache_text_encoder_outputs else 0,
                "network_dim": network_dim,
                "network_alpha": network_alpha,
                "network_module": network_module,
                "network_args": network_args,
                "network_train_unet_only": text_encoder_lr == 0 or cache_text_encoder_outputs,
            },
            "optimizer_arguments": {
                "learning_rate": unet_lr,
                "lr_scheduler": lr_scheduler,
                "lr_scheduler_num_cycles": lr_scheduler_num_cycles if lr_scheduler == "cosine_with_restarts" else None,
                "lr_scheduler_power": lr_scheduler_power if lr_scheduler == "polynomial" else None,
                "lr_warmup_steps": lr_warmup_steps if lr_scheduler != "constant" else None,
                "optimizer_type": optimizer,
                "optimizer_args": optimizer_args if optimizer_args else None,
            },
            "training_arguments": {
                "pretrained_model_name_or_path": model_file,
                "vae": vae_file,
                "max_train_steps": max_train_steps,
                "max_train_epochs": max_train_epochs,
                "train_batch_size": train_batch_size,
                "seed": 42,
                "max_token_length": 225,
                "xformers": cross_attention == "xformers",
                "sdpa": cross_attention == "sdpa",
                "min_snr_gamma": min_snr_gamma if min_snr_gamma > 0 else None,
                "lowram": LOWRAM,
                "no_half_vae": True,
                "gradient_checkpointing": True,
                "gradient_accumulation_steps": 1,
                "max_data_loader_n_workers": 8,
                "persistent_data_loader_workers": True,
                "mixed_precision": mixed_precision,
                "full_bf16": mixed_precision == "bf16",
                "cache_latents": cache_latents,
                "cache_latents_to_disk": cache_latents_to_drive,
                "cache_text_encoder_outputs": cache_text_encoder_outputs,
                "min_timestep": 0,
                "max_timestep": 1000,
                "prior_loss_weight": 1.0,
                "multires_noise_iterations": 6 if multinoise else None,
                "multires_noise_discount": 0.3 if multinoise else None,
            },
            "saving_arguments": {
                "save_precision": "fp16",
                "save_model_as": "safetensors",
                "save_every_n_epochs": save_every_n_epochs,
                "save_last_n_epochs": keep_only_last_n_epochs,
                "output_name": project_name,
                "output_dir": output_folder,
                "log_prefix": project_name,
                "logging_dir": log_folder,
                "wandb_api_key": wandb_key if wandb_key else None,
                "log_with": "wandb" if wandb_key else None,
            }
        }

        for key in config_dict:
            if isinstance(config_dict[key], dict):
                config_dict[key] = {k: v for k, v in config_dict[key].items() if v is not None}

        with open(config_file, "w") as f:
            f.write(toml.dumps(config_dict))
        print(f"\n📄 Config saved to {config_file}")

    if override_dataset_config_file:
        dataset_config_file = override_dataset_config_file
        print(f"⭕ Using custom dataset config file {dataset_config_file}")
    else:
        dataset_config_dict = {
            "general": {
                "resolution": resolution,
                "shuffle_caption": shuffle_caption and not cache_text_encoder_outputs,
                "keep_tokens": keep_tokens,
                "flip_aug": False,
                "caption_extension": caption_extension,
                "enable_bucket": True,
                "bucket_no_upscale": False,
                "bucket_reso_steps": 64,
                "min_bucket_reso": 256,
                "max_bucket_reso": 4096,
            },
            "datasets": toml.loads(custom_dataset)["datasets"] if custom_dataset else [
                {
                    "subsets": [
                        {
                            "num_repeats": num_repeats,
                            "image_dir": images_folder,
                            "class_tokens": None if caption_extension else project_name
                        }
                    ]
                }
            ]
        }

        for key in dataset_config_dict:
            if isinstance(dataset_config_dict[key], dict):
                dataset_config_dict[key] = {k: v for k, v in dataset_config_dict[key].items() if v is not None}

        with open(dataset_config_file, "w") as f:
            f.write(toml.dumps(dataset_config_dict))
        print(f"📄 Dataset config saved to {dataset_config_file}")

def download_model(load_diffusers):
    global old_model_url, model_url, model_file, vae_url, vae_file
    real_model_url = model_url.strip()

    if load_diffusers:
        if 'huggingface.co' in real_model_url:
            match = re.search(r'huggingface\.co/([^/]+)/([^/]+)', real_model_url)
            if match:
                username = match.group(1)
                model_name = match.group(2)
                model_file = f"{username}/{model_name}"
                fs = HfFileSystem()
                existing_folders = set(fs.ls(model_file, detail=False))
                necessary_folders = [ "scheduler", "text_encoder", "text_encoder_2", "tokenizer", "tokenizer_2", "unet", "vae" ]
                if all(f"{model_file}/{folder}" in existing_folders for folder in necessary_folders):
                    print("🍃 Diffusers model identified.")
                    return True
        print("🍃 Failed to load Diffusers model.")

    if not model_file:
        if real_model_url.lower().endswith((".ckpt", ".safetensors")):
            model_file = f"/content{real_model_url[real_model_url.rfind('/'):]}"
        else:
            model_file = "/content/downloaded_model.safetensors"
            if os.path.exists(model_file):
                !rm "{model_file}"

    if m := re.search(r"(?:https?://)?(?:www\.)?huggingface\.co/[^/]+/[^/]+/blob", model_url):
        real_model_url = real_model_url.replace("blob", "resolve")
    elif m := re.search(r"(?:https?://)?(?:www\\.)?civitai\.com/models/([0-9]+)(/[A-Za-z0-9-_]+)?", model_url):
        if m.group(2):
            model_file = f"/content{m.group(2)}.safetensors"
        if m := re.search(r"modelVersionId=([0-9]+)", model_url):
            real_model_url = f"https://civitai.com/api/download/models/{m.group(1)}"
        else:
            raise ValueError("optional_custom_training_model_url contains a civitai link, but the link doesn't include a modelVersionId. You can also right click the download button to copy the direct download link.")

    !aria2c "{real_model_url}" --console-log-level=warn -c -s 16 -x 16 -k 10M -d / -o "{model_file}"

    if not os.path.exists(vae_file):
        !aria2c "{vae_url}" --console-log-level=warn -c -s 16 -x 16 -k 10M -d / -o "{vae_file}"

    if model_file.lower().endswith(".safetensors"):
        from safetensors.torch import load_file as load_safetensors
        try:
            test = load_safetensors(model_file)
            del test
        except:
            #if "HeaderTooLarge" in str(e):
            new_model_file = os.path.splitext(model_file)[0]+".ckpt"
            !mv "{model_file}" "{new_model_file}"
            model_file = new_model_file
            print(f"Renamed model to {os.path.splitext(model_file)[0]}.ckpt")

    if model_file.lower().endswith(".ckpt"):
        from torch import load as load_ckpt
        try:
            test = load_ckpt(model_file)
            del test
        except:
            return False

    return True

def main(b):
    global project_name, folder_structure, training_model, optional_custom_training_model_url
    global training_model, training_model, model_url, vae_file, model_file, resolution, caption_extension
    global shuffle_tags, shuffle_caption, activation_tags
    global num_repeats, preferred_unit, how_many, max_train_epochs, max_train_steps
    global max_train_epochs, max_train_steps, preferred_unit, how_many
    global save_every_n_epochs, keep_only_last_n_epochs, unet_lr, text_encoder_lr
    global lr_scheduler, lr_scheduler_number, lr_warmup_ratio, lr_warmup_steps
    global min_snr_gamma, multinoise, lora_type, network_dim, conv_alpha
    global network_module, network_args, train_batch_size, cross_attention 
    global mixed_precision, cache_latents, cache_latents_to_drive
    global cache_text_encoder_outputs, optimizer, optimizer_args, recommended_values
    global images_folder, root_dir, network_alpha, wandb_key, vae_url, keep_tokens
    global lr_scheduler_num_cycles, lr_scheduler_power, output_folder
    
    project_name = project_name_widget.value
    project_name = project_name.strip()
    
    folder_structure = folder_structure_widget.value
    training_model = training_model_widget.value
    optional_custom_training_model_url = optional_custom_training_model_url_widget.value
    load_diffusers = load_diffusers_widget.value
    wandb_key = wandb_key_widget.value
    
    if optional_custom_training_model_url:
        model_url = optional_custom_training_model_url
    elif "Pony" in training_model:
        if load_diffusers:
            model_url = "https://huggingface.co/hollowstrawberry/67AB2F"
        else:
            model_url = "https://civitai.com/api/download/models/290640"
        model_file = "/content/ponyDiffusionV6XL.safetensors"
    elif "Animagine" in training_model:
        if load_diffusers:
            model_url = "https://huggingface.co/cagliostrolab/animagine-xl-3.0"
        else:
            model_url = "https://civitai.com/api/download/models/293564"
        model_file = "/content/animagineXLV3.safetensors"
    else:
        if load_diffusers:
            model_url = "https://huggingface.co/stabilityai/stable-diffusion-xl-base-1.0/"
        else:
            model_url = "https://huggingface.co/stabilityai/stable-diffusion-xl-base-1.0/resolve/main/sd_xl_base_1.0.safetensors"
    
    if load_diffusers:
        vae_file = "stabilityai/sdxl-vae"
    else:
        vae_url = "https://huggingface.co/stabilityai/sdxl-vae/resolve/main/sdxl_vae.safetensors"
        vae_file = "/content/sdxl_vae.safetensors"
    
    resolution = resolution_widget.value
    caption_extension = caption_extension_widget.value

    shuffle_tags = shuffle_tags_widget.value
    shuffle_caption = shuffle_tags
    activation_tags = activation_tags_widget.value
    keep_tokens = int(activation_tags)
    
    num_repeats = num_repeats_widget.value
    preferred_unit = preferred_unit_widget.value
    how_many = how_many_widget.value
    
    max_train_epochs = how_many if preferred_unit == "Epochs" else None
    max_train_steps = how_many if preferred_unit == "Steps" else None
    
    save_every_n_epochs = save_every_n_epochs_widget.value
    keep_only_last_n_epochs = keep_only_last_n_epochs_widget.value
    
    if not save_every_n_epochs:
        save_every_n_epochs = max_train_epochs
    if not keep_only_last_n_epochs:
        keep_only_last_n_epochs = max_train_epochs
    
    unet_lr = unet_lr_widget.value
    text_encoder_lr = text_encoder_lr_widget.value
    
    lr_scheduler = lr_scheduler_widget.value
    lr_scheduler_number = lr_scheduler_number_widget.value
    
    lr_warmup_ratio = lr_warmup_ratio_widget.value
    lr_warmup_steps = 0
    
    min_snr_gamma = min_snr_gamma_widget.value
    
    multinoise = multinoise_widget.value
    
    lora_type = lora_type_widget.value
    
    network_dim = network_dim_widget.value
    network_alpha = network_alpha_widget.value
    
    conv_dim = conv_dim_widget.value
    conv_alpha = conv_alpha_widget.value
    
    network_module = "networks.lora"
    network_args = None
    if lora_type.lower() == "locon":
        network_args = [f"conv_dim={conv_dim}", f"conv_alpha={conv_alpha}"]

    train_batch_size = train_batch_size_widget.value
    cross_attention = cross_attention_widget.value
    
    mixed_precision = mixed_precision_widget.value
    
    cache_latents = cache_latents_widget.value
    cache_latents_to_drive = cache_latents_to_drive_widget.value
    
    cache_text_encoder_outputs = cache_text_encoder_outputs_widget.value
    
    optimizer = optimizer_widget.value
    
    optimizer_args = optimizer_args_widget.value
    optimizer_args = [a.strip() for a in optimizer_args.split(' ') if a]
    
    recommended_values = recommended_values_widget.value
    
    if any(opt in optimizer.lower() for opt in ["dadapt", "prodigy"]):
        if recommended_values:
            unet_lr = 0.75
            text_encoder_lr = 0.75
            network_alpha = network_dim
    elif "CAME" in optimizer:
        optimizer = "CAME"
        lr_scheduler = "REX"
        if recommended_values:
            unet_lr = 1e-4
            text_encoder_lr = 1e-6
            for i in range(len(optimizer_args)):
                if "betas" in optimizer_args[i]:
                    optimizer_args.pop(i)
                    break
    
    lr_scheduler_num_cycles = lr_scheduler_number if lr_scheduler == "cosine_with_restarts" else 0
    lr_scheduler_power = lr_scheduler_number if lr_scheduler == "polynomial" else 0

    root_dir = "/content" if COLAB else pathlib.Path.home() / "Loras"
    deps_dir = os.path.join(root_dir, "deps")
    repo_dir = os.path.join(root_dir, "sd-scripts")

    global images_folder
    if "/Loras" in folder_structure:
        main_dir            = os.path.join(root_dir, "drive/MyDrive/Loras") if COLAB else root_dir
        log_folder        = os.path.join(main_dir, "_logs")
        config_folder = os.path.join(main_dir, project_name)
        images_folder = os.path.join(main_dir, project_name, "dataset")
        output_folder = os.path.join(main_dir, project_name, "output")
    else:
        main_dir            = os.path.join(root_dir, "drive/MyDrive/lora_training") if COLAB else root_dir
        images_folder = os.path.join(main_dir, "datasets", project_name)
        output_folder = os.path.join(main_dir, "output", project_name)
        config_folder = os.path.join(main_dir, "config", project_name)
        log_folder        = os.path.join(main_dir, "log")
    
    global config_file, dataset_config_file, accelerate_config_file
    
    config_file = os.path.join(config_folder, "training_config.toml")
    dataset_config_file = os.path.join(config_folder, "dataset_config.toml")
    accelerate_config_file = os.path.join(repo_dir, "accelerate_config/config.yaml")
    
    global dependencies_installed

    if COLAB and not os.path.exists('/content/drive'):
        from google.colab import drive
        print("📂 Connecting to Google Drive...")
        drive.mount('/content/drive')

    for dir in (main_dir, deps_dir, repo_dir, log_folder, images_folder, output_folder, config_folder):
        os.makedirs(dir, exist_ok=True)

    if not validate_dataset():
        return

    if not dependencies_installed:
        print("\n🏭 Installing dependencies...\n")
        t0 = time()
        install_dependencies(root_dir, repo_dir)
        t1 = time()
        dependencies_installed = True
        print(f"\n✅ Installation finished in {int(t1-t0)} seconds.")
    else:
        print("\n✅ Dependencies already installed.")

    if old_model_url != model_url or not model_file or not os.path.exists(model_file):
        print("\n🔄 Downloading model...")
        if not download_model(load_diffusers):
            print("\n💥 Error: The model you selected is invalid or corrupted, or couldn't be downloaded. You can use a civitai or huggingface link, or any direct download link.")
            return
        print()
    else:
        print("\n🔄 Model already downloaded.\n")

    create_config(images_folder, output_folder, log_folder)

    print("\n⭐ Starting trainer...\n")
    os.chdir(repo_dir)

    !accelerate launch --quiet --config_file={accelerate_config_file} --num_cpu_threads_per_process=1 train_network_xl_wrapper.py --dataset_config={dataset_config_file} --config_file={config_file}

    if not get_ipython().__dict__['user_ns']['_exit_code']:
        display(Markdown("### ✅ Done! [Go download your Lora from Google Drive](https://drive.google.com/drive/my-drive)\n"
                                         "### There will be several files, you should try the latest version (the file with the largest number next to it)"))

run_btn = widgets.Button(
    description='Start',
    disabled=False,
    button_style='success', 
    tooltip='Start',
    icon='play' 
)

display(
    widgets.HTML('<h2>▶️ Ready</h2>'),
    widgets.Label('You can now run this cell to cook your Lora. Good luck!'),
    run_btn
)

run_btn.on_click(main)

HTML(value='<h2>▶️ Ready</h2>')

Label(value='You can now run this cell to cook your Lora. Good luck!')

Button(button_style='success', description='Start', icon='play', style=ButtonStyle(), tooltip='Start')


💿 Checking dataset...
📁C:\Users\N\Loras\spy_x_family\dataset
📈 Found 1068 images with 2 repeats, equaling 2136 steps.
📉 Divide 2136 steps by 4 batch size to get 534.0 steps per epoch.
🔮 There will be 10 epochs, for around 5340 total training steps.

✅ Dependencies already installed.

🔄 Downloading model...
🍃 Diffusers model identified.



NameError: name 'log_folder' is not defined

## *️⃣ Extras

You can run these before starting the training.

### 📚 Multiple folders in dataset
Below is a template allowing you to define multiple folders in your dataset. You must include the location of each folder and you can set different number of repeats for each one. To add more folders simply copy and paste the sections starting with `[[datasets.subsets]]`.

When enabling this, the number of repeats set in the main cell will be ignored, and the main folder set by the project name will also be ignored.

You can make one of them a regularization folder by adding `is_reg = true`  
You can also set different `keep_tokens`, `flip_aug`, etc.

In [25]:
custom_dataset = """
[[datasets]]

[[datasets.subsets]]
image_dir = "/content/drive/MyDrive/Loras/example/dataset/good_images"
num_repeats = 3

[[datasets.subsets]]
image_dir = "/content/drive/MyDrive/Loras/example/dataset/normal_images"
num_repeats = 1

"""

In [26]:
custom_dataset = None

In [27]:
#@markdown ### 📂 Unzip dataset
#@markdown It's much slower to upload individual files to your Drive, so you may want to upload a zip if you have your dataset in your computer.
zip = "/content/drive/MyDrive/my_dataset.zip" #@param {type:"string"}
extract_to = "/content/drive/MyDrive/Loras/example/dataset" #@param {type:"string"}

import os, zipfile

if not os.path.exists('/content/drive'):
    from google.colab import drive
    print("📂 Connecting to Google Drive...")
    drive.mount('/content/drive')

os.makedirs(extract_to, exist_ok=True)

with zipfile.ZipFile(zip, 'r') as f:
    f.extractall(extract_to)

print("✅ Done")


ModuleNotFoundError: No module named 'google.colab'

fatal: destination path 'C:\Users\N\Loras\sd-scripts' already exists and is not an empty directory.
HEAD is now at bfb352b change huber_schedule from `exponential` to `snr`


HEAD is now at bfb352b change huber_schedule from `exponential` to `snr`


'wget' is not recognized as an internal or external command,
operable program or batch file.
'wget' is not recognized as an internal or external command,
operable program or batch file.


Obtaining file:///C:/Users/N/Loras/sd-scripts (from -r requirements.txt (line 42))
  Installing build dependencies: started
  Installing build dependencies: finished with status 'done'
  Checking if build backend supports build_editable: started
  Checking if build backend supports build_editable: finished with status 'done'
  Getting requirements to build editable: started
  Getting requirements to build editable: finished with status 'done'
  Preparing editable metadata (pyproject.toml): started
  Preparing editable metadata (pyproject.toml): finished with status 'done'
  Building editable for library (pyproject.toml): started
  Building editable for library (pyproject.toml): finished with status 'done'
  Created wheel for library: filename=library-0.0.0-0.editable-py3-none-any.whl size=6783 sha256=cb783fbbecc05671331d7ce9c862030df310a6846c7cde75de27a076d43cabe5
  Stored in directory: C:\Users\N\AppData\Local\Temp\pip-ephem-wheel-cache-4gwq0js2\wheels\46\d5\d3\6e946c942d8d8bb1190d4b4


[notice] A new release of pip is available: 24.0 -> 24.1.2
[notice] To update, run: python.exe -m pip install --upgrade pip
'sed' is not recognized as an internal or external command,
operable program or batch file.
'sed' is not recognized as an internal or external command,
operable program or batch file.
'sed' is not recognized as an internal or external command,
operable program or batch file.


In [ ]:
#@markdown ### 🔢 Count datasets
#@markdown Google Drive makes it impossible to count the files in a folder, so this will show you the file counts in all folders and subfolders.
folder = "/content/drive/MyDrive/Loras" #@param {type:"string"}

import os
from google.colab import drive

if not os.path.exists('/content/drive'):
    print("📂 Connecting to Google Drive...\n")
    drive.mount('/content/drive')

tree = {}
exclude = ("_logs", "/output")
for i, (root, dirs, files) in enumerate(os.walk(folder, topdown=True)):
    dirs[:] = [d for d in dirs if all(ex not in d for ex in exclude)]
    images = len([f for f in files if f.lower().endswith((".png", ".jpg", ".jpeg"))])
    captions = len([f for f in files if f.lower().endswith(".txt")])
    others = len(files) - images - captions
    path = root[folder.rfind("/")+1:]
    tree[path] = None if not images else f"{images:>4} images | {captions:>4} captions |"
    if tree[path] and others:
        tree[path] += f" {others:>4} other files"

pad = max(len(k) for k in tree)
print("\n".join(f"📁{k.ljust(pad)} | {v}" for k, v in tree.items() if v))


# 📈 Plot training results
You can do this after running the trainer. You don't need this unless you know what you're doing.  
The first cell below may fail to load all your logs. Keep trying the second cell until all data has loaded.

In [ ]:
%load_ext tensorboard
%tensorboard --logdir={log_folder}/

In [ ]:
from tensorboard import notebook
notebook.display(port=6006, height=800)